# 🔮 CardioFusion: Interactive Prediction Widget

## 📋 Overview
This notebook provides an **interactive prediction interface** using Jupyter widgets. Enter patient information and get real-time cardiovascular disease risk predictions.

### ✨ Features
- 🎨 Interactive form controls
- 🔄 Real-time predictions
- 📊 Visual risk assessment
- 🔍 SHAP explainability
- 💡 Clinical recommendations

---

In [1]:
# Import libraries
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Wedge
import warnings
import sys
from pathlib import Path
warnings.filterwarnings('ignore')

# Add parent directory to path to import utils
sys.path.insert(0, str(Path('..').resolve()))

# Import utility modules
from src.utils.model_utils import ModelPredictor
from src.utils.data_validator import DataValidator
from src.utils.shap_explainer import SHAPExplainer

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")
print("🎨 Widget interface ready to use!")

✅ All libraries imported successfully!
🎨 Widget interface ready to use!


## 🔄 Load Models

In [2]:
print("📂 Loading trained models...\n")

# Initialize model predictor (models folder is at ../models from notebooks/)
predictor = ModelPredictor('../models')
success = predictor.load_models()

if success:
    print("\n✅ All models loaded successfully!")
    print(f"📊 Available models: {', '.join(predictor.get_available_models())}")
else:
    print("❌ Failed to load models. Please ensure models are trained.")

# Load background data for SHAP from Data folder
try:
    train_data = pd.read_csv('../Data/train_data.csv')
    X_train = train_data.drop('Heart_Disease', axis=1)
    background_data = X_train.sample(min(100, len(X_train)), random_state=42)
    print("✅ Background data loaded for SHAP analysis")
except:
    background_data = None
    print("⚠️ Background data not available - SHAP analysis will be limited")

📂 Loading trained models...

⚠️ Preprocessing components not found
  📊 Loaded: Logistic Regression
  📊 Loaded: Decision Tree
  📊 Loaded: Random Forest
✅ Successfully loaded 3 models

✅ All models loaded successfully!
📊 Available models: Logistic Regression, Decision Tree, Random Forest
⚠️ Background data not available - SHAP analysis will be limited


## 🎨 Create Interactive Form

In [3]:
# Styled HTML header
header_html = """
<div style="background: linear-gradient(135deg, #1e3a8a 0%, #3b82f6 100%);
            padding: 30px;
            border-radius: 10px;
            margin-bottom: 20px;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
    <h1 style="color: white; margin: 0; font-size: 2.5em;">🩺 CardioFusion</h1>
    <p style="color: #e0e7ff; margin: 10px 0 0 0; font-size: 1.2em;">
        Interactive Cardiovascular Risk Assessment
    </p>
</div>
"""
display(HTML(header_html))

# ============================================
# SECTION 1: Demographics
# ============================================

display(HTML('<h2 style="color: #1e3a8a; border-bottom: 3px solid #3b82f6; padding-bottom: 10px;">👤 Demographic Information</h2>'))

age_categories = ['18-24', '25-29', '30-34', '35-39', '40-44', '45-49',
                 '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80+']

age_widget = widgets.Dropdown(
    options=age_categories,
    value='50-54',
    description='Age Group:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='400px')
)

sex_widget = widgets.RadioButtons(
    options=['Male', 'Female'],
    description='Sex:',
    style={'description_width': '150px'}
)

demo_box = widgets.HBox([age_widget, sex_widget])
display(demo_box)

# ============================================
# SECTION 2: Physical Measurements
# ============================================

display(HTML('<br><h2 style="color: #1e3a8a; border-bottom: 3px solid #3b82f6; padding-bottom: 10px;">📏 Physical Measurements</h2>'))

height_widget = widgets.IntSlider(
    value=170,
    min=100,
    max=250,
    step=1,
    description='Height (cm):',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px'),
    readout=True
)

weight_widget = widgets.FloatSlider(
    value=75.0,
    min=30.0,
    max=300.0,
    step=0.5,
    description='Weight (kg):',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px'),
    readout=True
)

bmi_display = widgets.HTML(value='<b>BMI:</b> 25.9')

def update_bmi(*args):
    height_m = height_widget.value / 100
    bmi = weight_widget.value / (height_m ** 2)
    
    # Color code BMI
    if bmi < 18.5:
        color = '#3b82f6'
        category = 'Underweight'
    elif bmi < 25:
        color = '#059669'
        category = 'Normal'
    elif bmi < 30:
        color = '#d97706'
        category = 'Overweight'
    else:
        color = '#dc2626'
        category = 'Obese'
    
    bmi_display.value = f'<div style="background: {color}; color: white; padding: 10px; border-radius: 5px; text-align: center; width: 200px;"><b>BMI: {bmi:.1f}</b><br><small>{category}</small></div>'

height_widget.observe(update_bmi, 'value')
weight_widget.observe(update_bmi, 'value')

display(height_widget)
display(weight_widget)
display(bmi_display)

# ============================================
# SECTION 3: Lifestyle Factors
# ============================================

display(HTML('<br><h2 style="color: #1e3a8a; border-bottom: 3px solid #3b82f6; padding-bottom: 10px;">🏃 Lifestyle Factors</h2>'))

exercise_widget = widgets.RadioButtons(
    options=['Yes', 'No'],
    value='Yes',
    description='Regular Exercise:',
    style={'description_width': '150px'}
)

smoking_widget = widgets.RadioButtons(
    options=['No', 'Yes'],
    value='No',
    description='Smoking History:',
    style={'description_width': '150px'}
)

lifestyle_box1 = widgets.HBox([exercise_widget, smoking_widget])
display(lifestyle_box1)

alcohol_widget = widgets.IntSlider(
    value=0,
    min=0,
    max=30,
    description='Alcohol (units/mo):',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px')
)

fruit_widget = widgets.IntSlider(
    value=30,
    min=0,
    max=120,
    description='Fruit (servings/mo):',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px')
)

vegetables_widget = widgets.IntSlider(
    value=12,
    min=0,
    max=128,
    description='Vegetables (servings/mo):',
    style={'description_width': '180px'},
    layout=widgets.Layout(width='500px')
)

fried_potato_widget = widgets.IntSlider(
    value=4,
    min=0,
    max=128,
    description='Fried Potato (servings/mo):',
    style={'description_width': '180px'},
    layout=widgets.Layout(width='500px')
)

display(alcohol_widget)
display(fruit_widget)
display(vegetables_widget)
display(fried_potato_widget)

# ============================================
# SECTION 4: Health Status
# ============================================

display(HTML('<br><h2 style="color: #1e3a8a; border-bottom: 3px solid #3b82f6; padding-bottom: 10px;">🏥 Health Status</h2>'))

general_health_widget = widgets.SelectionSlider(
    options=['Poor', 'Fair', 'Good', 'Very Good', 'Excellent'],
    value='Good',
    description='General Health:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px')
)

checkup_widget = widgets.Dropdown(
    options=['Within the past year', 'Within the past 2 years',
             'Within the past 5 years', '5 or more years ago', 'Never'],
    value='Within the past year',
    description='Last Checkup:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px')
)

diabetes_widget = widgets.Dropdown(
    options=['No', 'Yes', 'No, pre-diabetes or borderline diabetes',
             'Yes, but female told only during pregnancy'],
    value='No',
    description='Diabetes:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px')
)

display(general_health_widget)
display(checkup_widget)
display(diabetes_widget)

display(HTML('<br><h3>Medical Conditions:</h3>'))

depression_widget = widgets.Checkbox(value=False, description='Depression')
arthritis_widget = widgets.Checkbox(value=False, description='Arthritis')
skin_cancer_widget = widgets.Checkbox(value=False, description='Skin Cancer')
other_cancer_widget = widgets.Checkbox(value=False, description='Other Cancer')

conditions_box = widgets.HBox([
    widgets.VBox([depression_widget, arthritis_widget]),
    widgets.VBox([skin_cancer_widget, other_cancer_widget])
])
display(conditions_box)

# ============================================
# SECTION 5: View Mode Toggle
# ============================================

display(HTML('<br><h2 style="color: #1e3a8a; border-bottom: 3px solid #3b82f6; padding-bottom: 10px;">⚙️ Settings</h2>'))

view_mode_widget = widgets.RadioButtons(
    options=['Simple View', 'Detailed Analysis'],
    value='Simple View',
    description='Detail Level:',
    style={'description_width': '150px'}
)

display(view_mode_widget)

print("\n✅ Interactive form created!")
print("📝 Fill in the patient information above and run the next cell to get predictions.")

IntSlider(value=170, description='Height (cm):', layout=Layout(width='500px'), max=250, min=100, style=SliderS…

FloatSlider(value=75.0, description='Weight (kg):', layout=Layout(width='500px'), max=300.0, min=30.0, step=0.…

HTML(value='<b>BMI:</b> 25.9')

IntSlider(value=0, description='Alcohol (units/mo):', layout=Layout(width='500px'), max=30, style=SliderStyle(…

IntSlider(value=30, description='Fruit (servings/mo):', layout=Layout(width='500px'), max=120, style=SliderSty…

...

SelectionSlider(description='General Health:', index=2, layout=Layout(width='500px'), options=('Poor', 'Fair',…

Dropdown(description='Last Checkup:', layout=Layout(width='500px'), options=('Within the past year', 'Within t…

Dropdown(description='Diabetes:', layout=Layout(width='500px'), options=('No', 'Yes', 'No, pre-diabetes or bor…

RadioButtons(description='Detail Level:', options=('Simple View', 'Detailed Analysis'), style=DescriptionStyle…


✅ Interactive form created!
📝 Fill in the patient information above and run the next cell to get predictions.


## 🔬 Generate Prediction

In [ ]:
# Collect patient data
patient_data = {
    'age_category': age_widget.value,
    'sex': sex_widget.value,
    'height_cm': height_widget.value,
    'weight_kg': weight_widget.value,
    'bmi': weight_widget.value / ((height_widget.value/100) ** 2),
    'exercise': exercise_widget.value,
    'smoking_history': smoking_widget.value,
    'alcohol_consumption': alcohol_widget.value,
    'fruit_consumption': fruit_widget.value,
    'green_vegetables_consumption': vegetables_widget.value,
    'fried_potato_consumption': fried_potato_widget.value,
    'general_health': general_health_widget.value,
    'checkup': checkup_widget.value,
    'diabetes': diabetes_widget.value,
    'depression': 'Yes' if depression_widget.value else 'No',
    'arthritis': 'Yes' if arthritis_widget.value else 'No',
    'skin_cancer': 'Yes' if skin_cancer_widget.value else 'No',
    'other_cancer': 'Yes' if other_cancer_widget.value else 'No'
}

# Validate and preprocess
validator = DataValidator()
is_valid, errors, warnings = validator.validate_input(patient_data)

if not is_valid:
    print("❌ VALIDATION ERRORS:")
    for error in errors:
        print(f"  {error}")
else:
    if warnings:
        print("⚠️ WARNINGS:")
        for warning in warnings:
            print(f"  {warning}")
        print()
    
    # Preprocess
    input_df = validator.preprocess_for_model(patient_data)
    
    # Make prediction
    print("🔄 Analyzing patient data...\n")
    prediction = predictor.predict(input_df)
    
    # Get risk category
    risk_pct = prediction['risk_percentage']
    category, emoji, color = predictor.get_risk_category(risk_pct)
    
    # ============================================
    # DISPLAY RESULTS
    # ============================================
    
    # Header
    result_header = f"""
    <div style="background: linear-gradient(135deg, {color} 0%, {color}dd 100%);
                padding: 30px;
                border-radius: 10px;
                margin: 20px 0;
                box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
        <h1 style="color: white; margin: 0; font-size: 3em;">{emoji} {category}</h1>
        <h2 style="color: white; margin: 10px 0 0 0;">Risk Score: {risk_pct:.1f}%</h2>
    </div>
    """
    display(HTML(result_header))
    
    # Create visualization
    if view_mode_widget.value == 'Simple View':
        # Simple gauge chart
        fig = go.Figure(go.Indicator(
            mode="gauge+number",
            value=risk_pct,
            domain={'x': [0, 1], 'y': [0, 1]},
            title={'text': "Risk Score", 'font': {'size': 24}},
            number={'suffix': "%", 'font': {'size': 48}},
            gauge={
                'axis': {'range': [None, 100]},
                'bar': {'color': color},
                'steps': [
                    {'range': [0, 30], 'color': '#d1fae5'},
                    {'range': [30, 50], 'color': '#fef3c7'},
                    {'range': [50, 70], 'color': '#fed7aa'},
                    {'range': [70, 100], 'color': '#fecaca'}
                ],
                'threshold': {
                    'line': {'color': "black", 'width': 4},
                    'thickness': 0.75,
                    'value': risk_pct
                }
            }
        ))
        
        fig.update_layout(
            height=350,
            margin=dict(l=20, r=20, t=60, b=20)
        )
        
        fig.show()
        
    else:
        # Detailed view with individual model predictions
        print("\n📊 INDIVIDUAL MODEL PREDICTIONS")
        print("="*50)
        
        if 'individual_models' in prediction:
            models_data = []
            for model_name, results in prediction['individual_models'].items():
                models_data.append({
                    'Model': model_name,
                    'Risk Probability': f"{results['probability_disease']*100:.1f}%",
                    'Weight': results['weight']
                })
            
            models_df = pd.DataFrame(models_data)
            display(models_df)
        
        # Risk gauge
        fig = go.Figure(go.Indicator(
            mode="gauge+number",
            value=risk_pct,
            domain={'x': [0, 1], 'y': [0, 1]},
            title={'text': "Risk Score", 'font': {'size': 24}},
            number={'suffix': "%", 'font': {'size': 48}},
            gauge={
                'axis': {'range': [None, 100]},
                'bar': {'color': color},
                'steps': [
                    {'range': [0, 30], 'color': '#d1fae5'},
                    {'range': [30, 50], 'color': '#fef3c7'},
                    {'range': [50, 70], 'color': '#fed7aa'},
                    {'range': [70, 100], 'color': '#fecaca'}
                ],
                'threshold': {
                    'line': {'color': "black", 'width': 4},
                    'thickness': 0.75,
                    'value': risk_pct
                }
            }
        ))
        
        fig.update_layout(height=350, margin=dict(l=20, r=20, t=60, b=20))
        fig.show()
        
        # Feature contributions (if available)
        if background_data is not None:
            try:
                print("\n🔍 TOP CONTRIBUTING FACTORS")
                print("="*50)
                
                from src.utils.shap_explainer import SHAPExplainer
                
                # Get best model for SHAP
                model = list(predictor.models.values())[0]
                explainer = SHAPExplainer(model, background_data)
                explanation = explainer.explain_prediction(input_df)
                
                if 'error' not in explanation:
                    print("\n🔴 Risk-Increasing Factors:")
                    for feature, value in explanation['top_positive'][:5]:
                        print(f"  • {feature.replace('_', ' ').title()}: +{value:.3f}")
                    
                    print("\n🟢 Risk-Decreasing Factors:")
                    for feature, value in explanation['top_negative'][:5]:
                        print(f"  • {feature.replace('_', ' ').title()}: {value:.3f}")
            except Exception as e:
                print(f"⚠️ SHAP analysis unavailable: {str(e)}")
    
    # Display metrics
    print("\n" + "="*50)
    print("📊 PREDICTION DETAILS")
    print("="*50)
    print(f"Prediction: {prediction['prediction_label']}")
    print(f"Confidence: {prediction['confidence']*100:.1f}%")
    print(f"Risk Score: {risk_pct:.1f}%")
    print(f"Category: {category}")
    
    # Medical disclaimer
    disclaimer = """
    <div style="background: #fef3c7; 
                border-left: 4px solid #d97706; 
                padding: 15px; 
                border-radius: 5px; 
                margin: 20px 0;">
        <strong>⚠️ Medical Disclaimer:</strong><br>
        This tool is for educational purposes only. It does not provide medical advice, 
        diagnosis, or treatment. Always consult with a qualified healthcare professional 
        for medical advice.
    </div>
    """
    display(HTML(disclaimer))

ValueError: not enough values to unpack (expected 4, got 3)

## 📝 Usage Instructions

### How to Use This Widget:

1. **Run all cells above** to create the interactive form
2. **Adjust the widgets** to enter patient information
3. **Select view mode**: Simple View or Detailed Analysis
4. **Run the prediction cell** to get results

### View Modes:

- **Simple View**: Quick risk assessment with gauge visualization
- **Detailed Analysis**: Complete analysis with SHAP explanations and recommendations

---

*CardioFusion - Professional ML for Heart Disease Prediction* 🩺